<a href="https://colab.research.google.com/github/otoperalias/Coyuntura/blob/main/clases/Tema3_V.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tema 3.5. Modelos SARIMA y SARIMAX.

In [ ]:
# Instalamos la librería pmdarima (la vamos a necesitar en esta sesión)
!pip install pmdarima --user # Si usamos Jupyter, quitamos el signo de exclamación

In [ ]:
# Ahora reiniciamos el entorno de ejecución

In [ ]:
# Importamos funciones y establecemos configuración general
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pmdarima as pm
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
plt.style.use('seaborn')
plt.rcParams["figure.figsize"] = [10,4]  # Default figure size

In [ ]:
# Generamos funciones que vamos a usar en el notebook
def DFtest(datos):
    print('Results of Dickey-Fuller Test (H0= NO estac.):')
    dftest = adfuller(datos, autolag='AIC')
    print('Test Statistic', dftest[0])
    print('p-value', dftest[1])
    print('#Lags Used', dftest[2])
    print('# obs', dftest[3])
    for k, v in dftest[4].items():
        print(k,v)

def forecast_accuracy(forecast, actual):
    rmse = np.sqrt(np.mean((forecast-actual)**2))
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    print({'rmse':rmse,'mape':mape,'corr':corr})

### Modelo SARIMA

In [ ]:
# Importamos base de datos (sobre ventas de medicamentos)
# https://github.com/otoperalias/Coyuntura/blob/main/clases/datos/medicamentos_ventas.csv
# clicar en 'raw' y luego clicar en botón derecho y 'guardar como' .csv
medic = pd.read_csv("medicamentos_ventas.csv",index_col=0,parse_dates=True)

#Creamos test and training dataset
train=medic[:-24]
test=medic[-24:]

In [ ]:
# GRÁFICO
fig, axes = plt.subplots(2, 1, figsize=(8,5), dpi=100, sharex=True)
# Diferenciación normal
axes[0].plot(medic, label='Serie original')
axes[0].plot(medic.diff(1), label='Diferenciación normal')
axes[0].set_title('Diferenciación normal')
axes[0].legend(loc='upper left', fontsize=10)
# Diferenciación estacional
axes[1].plot(medic, label='Serie original')
axes[1].plot(medic.diff(12), label='Diferenciación estacional', color='green')
axes[1].set_title('Diferenciación estacional')
plt.legend(loc='upper left', fontsize=10)
plt.suptitle('MEDICAMENTOS - VENTAS', fontsize=13)
plt.show()

Se puede observar que los picos estacionales no resultan afectados cuando se realiza la diferenciación normal, mientras que al diferenciar de manera estacional, desaparecen.

In [ ]:
# Dibujamos correlogramas y usamos el test DF
fig, ax = plt.subplots(2, figsize=(12,6))
ax[0] = plot_acf(medic.diff(12).dropna(), ax=ax[0], lags=28)
ax[1] = plot_pacf(medic.diff(12).dropna(), ax=ax[1], lags=28)
plt.show()
DFtest(medic.diff(12).dropna())

La serie es no estacionaria por lo que hay que tomar primeras diferencias

In [ ]:
fig, ax = plt.subplots(2, figsize=(12,6))
ax[0] = plot_acf(medic.diff(12).diff().dropna(), ax=ax[0], lags=28)
ax[1] = plot_pacf(medic.diff(12).diff().dropna(), ax=ax[1], lags=28)
plt.show()
DFtest(medic.diff(12).diff().dropna())

Ahora según el test de DF es claramente estacionaria, pero los gráficos indican que puede estar sobre diferenciada.  
Por tanto, tratamos de evitar esta doble diferenciación. Volvemos a mirar los gráficos de autocorrelación y correlación parcial de la serie con
diferenciación estacional.  
Vemos que el modelo podría ser: SARIMA(1,0,1)(1,1,0,12)

#### Procedemos a construir el modelo:
Para más información sobre modelos SARIMAX:
*https://www.statsmodels.org/stable/generated/statsmodels.tsa.statespace.sarimax.SARIMAX.html
*https://www.statsmodels.org/stable/examples/notebooks/generated/statespace_sarimax_stata.html


In [ ]:
# Especificación del modelo
mod = sm.tsa.statespace.SARIMAX(medic,order=(1,0,1),seasonal_order=(1,1,0,12), trend="c",
                                enforce_stationarity=False, enforce_invertibility=False)

In [ ]:
# Estimación del modelo y output de los resultados:v
results= mod.fit()
print(results.summary())   # vemos que la constante no es significativa y se podría quitar

In [ ]:
# Podemos dibujar los errores
results.plot_diagnostics()
plt.show()

#### ¿Cómo de preciso es nuestro modelo para predecir?

In [ ]:
# Construimos y estimamos el modelo con la "training dataset"
results = sm.tsa.statespace.SARIMAX(train,order=(1,0,1),seasonal_order=(1,1,0,12), trend="c",
                                    enforce_stationarity=False, enforce_invertibility=False).fit() # observar que obtengo el resultado directamente con .fit()
print(results.summary())

In [ ]:
# Realizo la predicción para el periodo de la "test dataset" y calculo su exactitud (acccuracy)
pred = results.get_prediction('2006-07-01','2008-06-01', dynamic=False)
forecast_accuracy(pred.predicted_mean, test.value)

In [ ]:
# Dibujamos la predicción:
pred_ci = pred.conf_int()
index_of_fc = pd.date_range(start="2006-07-01",end="2008-06-01", freq='MS')
# Creando las series para su representación gráfica
fitted_series = pd.Series(pred.predicted_mean, index=index_of_fc)
lower_series = pd.Series(pred_ci["lower value"], index=index_of_fc)
upper_series = pd.Series(pred_ci["upper value"], index=index_of_fc)
# Gráfico -
fig,ax=plt.subplots(figsize=(12,8))
ax.plot(train[-60:], label="Training period", color="b") #con [-60:] evito representar la serie completa
ax.plot(test, label="Actual values", color="k")
ax.plot(fitted_series, label="Predicted", color='r')
ax.fill_between(lower_series.index,
                 lower_series,
                 upper_series,
                 color='k', alpha=.15)
ax.legend()
ax.set_title("SARIMA - Predicción de ventas de medicamentos")
plt.show()

In [ ]:
# Y también predecimos hacia el futuro
mod = sm.tsa.statespace.SARIMAX(medic,order=(1,0,1),seasonal_order=(1,1,0,12), trend="c",
                                enforce_stationarity=False, enforce_invertibility=False)
pred = results.get_prediction(start="2008-07-01",end="2010-06-01")
pred_ci = pred.conf_int()
index_of_fc = pd.date_range(start="2008-07-01",end="2010-06-01", freq='MS')
# Creando las series para su representación gráfica
fitted_series = pd.Series(pred.predicted_mean, index=index_of_fc)
lower_series = pd.Series(pred_ci["lower value"], index=index_of_fc)
upper_series = pd.Series(pred_ci["upper value"], index=index_of_fc)
# Gráfico
fig,ax=plt.subplots(figsize=(12,8))
ax.plot(medic)
ax.plot(fitted_series, color='darkgreen')
ax.fill_between(lower_series.index,
                 lower_series,
                 upper_series,
                 color='k', alpha=.15)
ax.set_title("SARIMA - Predicción de ventas de medicamentos")
plt.show()

#### Construimos el modelo automáticamente

In [ ]:
# AHORA BUSCAMOS EL MODELO AUTOMÁTICAMENTE ------------------------------------
smodel = pm.auto_arima(train, start_p=0, start_q=0,
                         test='adf',
                         max_p=3, max_q=3, m=12,
                         start_P=0, seasonal=True,
                         trace=True,
                         error_action='ignore',
                         suppress_warnings=True,
                         stepwise=True)

In [ ]:
# Mostramos el output del modelo
print(smodel.summary())

In [ ]:
# Y predecimos
n_periods = 24
fitted, confint = smodel.predict(n_periods=n_periods, return_conf_int=True)
forecast_accuracy(fitted, test.value)

In [ ]:
# Gráfico de la predicción
index_of_fc = pd.date_range(start="2006-07-01",end="2008-06-01", freq='MS')
fitted_series = pd.Series(fitted, index=index_of_fc)
lower_series = pd.Series(confint[:, 0], index=index_of_fc)
upper_series = pd.Series(confint[:, 1], index=index_of_fc)
# Plot
fig,ax=plt.subplots(figsize=(12,8))
ax.plot(train[-60:], label="Training period", color="b") #con [-60:] evito representar la serie completa
ax.plot(test, label="Actual values", color="k")
ax.plot(fitted_series, label="Predicted", color='r')
ax.fill_between(lower_series.index,
                 lower_series,
                 upper_series,
                 color='k', alpha=.15)
ax.legend()
ax.set_title("SARIMA - Predicción de ventas de medicamentos")
plt.show()

Vemos que la predicción de ambos modelos es similar y nos gustaría que fuera más buena.  
Si tuviéramos un indicador correlacionado con las ventas de medicamentos y que estuviera disponible antes, sería muy útil para ayudarnos a predecir. Ese indicador sería la variable e**X**ógena del modelo *SARIMAX*.

### Modelo SARIMAX

In [ ]:
# Datos de la variable exógena: número de visitas a atención primaria
# https://github.com/otoperalias/Coyuntura/blob/main/clases/datos/medic_visitas_doctor.xlsx

visit = pd.read_excel(...) # https://raw.githubusercontent.com/otoperalias/Coyuntura/main
visit

Esta serie tienes 6 observaciones más, las cuales vamos utilizar para mejorar la predicción.

In [ ]:
# Visualizamos
fig,ax=plt.subplots(1,2,figsize=(10,4))
ax[0].plot(medic)
ax01 = ax[0].twinx()  # aquí se establece un segundo eje Y que comparte el mismo eje X
ax01.plot(visit,color='tab:red')
ax01.grid(False)
ax[1].scatter(medic.value,visit.visitas[:204]) # vemos que hay una correlación alta y por tanto puede usarse como variable exógena
plt.subplots_adjust(wspace=0.5)
plt.show()
print(np.corrcoef(medic.value,visit.visitas[:204])[0,1])

Vemos que las dos series están muy correlacionadas y por tanto la variable visitas puede usarse como eXógena.

#### Construimos el modelo:

In [ ]:
# Especificamos y estimamos el modelo
exog_in=visit[:-30] # -24 y -6 por las 6 obs extra ==> train y exog_in deben tener igual periodo (!)
results = sm.tsa.statespace.SARIMAX(train,exog=exog_in, order=(1,0,1),seasonal_order=(1,1,0,12), trend="c",
                                    enforce_stationarity=False, enforce_invertibility=False).fit()
print(results.summary())

#### Capacidad para predecir:

In [ ]:
# Capacidad para predecir
exog_out=visit['2006-07-01':'2008-06-01'] # por las seis observaciones extra para las cuales no sabemos el valor de medic
pred = results.get_prediction('2006-07-01','2008-06-01', exog=exog_out, dynamic=False)
forecast_accuracy(pred.predicted_mean, test.value)

In [ ]:
# Dibujamos la predicción:
pred_ci = pred.conf_int()
index_of_fc = pd.date_range(start="2006-07-01",end="2008-06-01", freq='MS')
# Creando las series para su representación gráfica
fitted_series = pd.Series(pred.predicted_mean, index=index_of_fc)
lower_series = pd.Series(pred_ci["lower value"], index=index_of_fc)
upper_series = pd.Series(pred_ci["upper value"], index=index_of_fc)
# Gráfico -
fig,ax=plt.subplots(figsize=(12,8))
ax.plot(train[-60:], label="Training period", color="b") #con [-60:] evito representar la serie completa
ax.plot(test, label="Actual values", color="k")
ax.plot(fitted_series, label="Predicted", color='r')
ax.fill_between(lower_series.index,
                 lower_series,
                 upper_series,
                 color='k', alpha=.15)
ax.legend()
ax.set_title("SARIMAX - Predicción de ventas de medicamentos \n usando visitas al doctor como variable exógena")
plt.show()

Y también predecimos hacia el futuro. Dado que visit llega a 2008-12-01 (mientras medic hasta 2008-06-01), predecimos seis meses.

In [ ]:
# Predecimos hacia el futuro:
results = sm.tsa.statespace.SARIMAX(medic,exog=visit[:-6], order=(1,0,1),seasonal_order=(1,1,0,12), trend="c",
                                    enforce_stationarity=False, enforce_invertibility=False).fit()
pred = results.get_prediction(start="2008-07-01",end="2008-12-01", exog=visit[-6:])
pred_ci = pred.conf_int()
index_of_fc = pd.date_range(start="2008-07-01",end="2008-12-01",freq='MS')
# Creando las series para su representación gráfica
fitted_series = pd.Series(pred.predicted_mean, index=index_of_fc)
lower_series = pd.Series(pred_ci["lower value"], index=index_of_fc)
upper_series = pd.Series(pred_ci["upper value"], index=index_of_fc)
# Gráfico
fig,ax=plt.subplots(figsize=(12,8))
ax.plot(medic[-60:])
ax.plot(fitted_series, color='darkgreen')
ax.fill_between(lower_series.index,
                 lower_series,
                 upper_series,
                 color='k', alpha=.15)
ax.set_title("SARIMAX - Predicción de ventas de medicamentos \n usando visitas al doctor como variable exógena")
plt.show()

#### Construimos el modelo automáticamente:

El método auto_arima también puede usarse para estimar el modelo SARIMAX óptimo usando el argumento **X**.

In [ ]:
smodel = pm.auto_arima(medic, start_p=0, start_q=0,
                         X=visit[:-6], test='adf',
                         max_p=3, max_q=3, m=12,
                         start_P=0, seasonal=True,
                         trace=True,
                         error_action='ignore',
                         suppress_warnings=True,
                         stepwise=True)
print(smodel.summary())

In [ ]:
# Y predecimos hacia el futuro
n_periods = 6
fitted, confint = smodel.predict(n_periods=n_periods, X=visit[-6:], return_conf_int=True)
index_of_fc = pd.date_range(start="2008-07-01",end="2008-12-01", freq='MS')
# Creando las series para su representación gráfica
fitted_series = pd.Series(fitted, index=index_of_fc)
lower_series = pd.Series(confint[:, 0], index=index_of_fc)
upper_series = pd.Series(confint[:, 1], index=index_of_fc)
# Gráfico
fig,ax=plt.subplots(figsize=(12,8))
ax.plot(medic[-60:], color="b") #con [-60:] evito representar la serie completa
ax.plot(fitted_series, label="Predicted", color='r')
ax.fill_between(lower_series.index,
                 lower_series,
                 upper_series,
                 color='k', alpha=.15)
ax.legend()
ax.set_title("SARIMAX - Predicción de ventas de medicamentos. \n usando visitas al médico como variable exógena")
plt.show()